In [32]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
h, w = 224, 224
vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(h, w, 3))

In [33]:
# Congelar las capas preentrenadas
for layer in vgg16.layers:
    layer.trainable = False

# Crear un nuevo modelo secuencial
model = Sequential()

# Agregar las capas de VGG16 al nuevo modelo
model.add(vgg16)

# Añadir capas personalizadas
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compilar el modelo
model.compile(optimizer="adam", loss='binary_crossentropy', metrics=['accuracy'])

In [34]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

training_dataset = train_datagen.flow_from_directory('../Datasets/cats_dogs/training_set',
                                                    target_size=(h, w),
                                                    batch_size=32,
                                                    class_mode='binary')

testing_dataset = test_datagen.flow_from_directory('../Datasets/cats_dogs/test_set',
                                                target_size=(h, w),
                                                batch_size=32,
                                                class_mode='binary')

# Entrenar el modelo
model.fit(training_dataset, steps_per_epoch=62, epochs=15, validation_data=testing_dataset, validation_steps=62)

Found 2000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Epoch 1/15
62/62 [==============================] - 1143s 19s/step - loss: 0.7010 - accuracy: 0.7561 - val_loss: 0.2606 - val_accuracy: 0.8775
Epoch 2/15
62/62 [==============================] - 1148s 19s/step - loss: 0.2391 - accuracy: 0.8948 - val_loss: 0.2177 - val_accuracy: 0.9078
Epoch 3/15
62/62 [==============================] - 1137s 18s/step - loss: 0.2077 - accuracy: 0.9121 - val_loss: 0.2789 - val_accuracy: 0.8841
Epoch 4/15
62/62 [==============================] - 1144s 19s/step - loss: 0.1610 - accuracy: 0.9385 - val_loss: 0.2586 - val_accuracy: 0.8982
Epoch 5/15
62/62 [==============================] - 1156s 19s/step - loss: 0.1501 - accuracy: 0.9380 - val_loss: 0.2557 - val_accuracy: 0.9002
Epoch 6/15
62/62 [==============================] - 1141s 19s/step - loss: 0.1487 - accuracy: 0.9390 - val_loss: 0.2538 - val_accuracy: 0.9037
Epoch 7/15
62/62 [==============================] - 1143s 

In [42]:
import numpy as np
import os
from tensorflow.keras.preprocessing import image

directory = '../Datasets/cats_dogs/test_image'

# Iterar sobre los archivos en el directorio
for filename in os.listdir(directory):
    # Combinar el directorio y el nombre del archivo
    filepath = os.path.join(directory, filename)
    
    # Cargar la imagen y ajustar su tamaño
    test_image = image.load_img(filepath, target_size=(h, w))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis=0)
    
    # Realizar la predicción
    result = model.predict(test_image)
    
    # Imprimir el resultado
    if result[0][0] == 1:
        prediction = 'Perro'
    elif result[0][0] == 0:
        prediction = 'Gato'
    
    print(f'La imagen {filename} es un/a {prediction}')

1/1 [==============================] - 0s 444ms/step
La imagen cat.4001.jpg es un/a Gato
1/1 [==============================] - 1s 600ms/step
La imagen cat.4002.jpg es un/a Perro
1/1 [==============================] - 0s 391ms/step
La imagen cat.4003.jpg es un/a Gato
1/1 [==============================] - 0s 409ms/step
La imagen cat.4004.jpg es un/a Gato
1/1 [==============================] - 1s 549ms/step
La imagen cat.4005.jpg es un/a Gato
1/1 [==============================] - 0s 452ms/step
La imagen cat.4006.jpg es un/a Gato
1/1 [==============================] - 0s 365ms/step
La imagen cat.4007.jpg es un/a Gato
1/1 [==============================] - 0s 373ms/step
La imagen cat.4008.jpg es un/a Perro
1/1 [==============================] - 0s 355ms/step
La imagen cat.4009.jpg es un/a Perro
1/1 [==============================] - 0s 356ms/step
La imagen cat.4010.jpg es un/a Perro
1/1 [==============================] - 0s 370ms/step
La imagen Hadelin_Dog.jpg es un/a Perro


In [45]:
from tensorflow.keras.models import load_model
model.save('./models/vvg16_dog_cat.h5')

In [46]:
model = load_model('./models/vvg16_dog_cat.h5')